task name: Implement ApplyCustomWordEmbedding
task description: Implement the ApplyCustomWordEmbedding example using ML.NET

In [ ]:
#r "nuget:Microsoft.ML, 1.6.0"

In [ ]:
using System;
using System.Collections.Generic;
using System.IO;
using Microsoft.ML;

In [ ]:
var mlContext = new MLContext();

In [ ]:
public class TextData
{
    public string Text { get; set; }
}

var emptySamples = new List<TextData>();
var emptyDataView = mlContext.Data.LoadFromEnumerable(emptySamples);

In [ ]:
var pathToCustomModel = @".\custommodel.txt";
using (StreamWriter file = new StreamWriter(pathToCustomModel, false))
{
    file.WriteLine("great 1.0 2.0 3.0");
    file.WriteLine("product -1.0 -2.0 -3.0");
    file.WriteLine("like -1 100.0 -100");
    file.WriteLine("buy 0 0 20");
}

In [ ]:
var textPipeline = mlContext.Transforms.Text.NormalizeText("Text")
    .Append(mlContext.Transforms.Text.TokenizeIntoWords("Tokens", "Text"))
    .Append(mlContext.Transforms.Text.ApplyWordEmbedding("Features", pathToCustomModel, "Tokens"));

In [ ]:
var textTransformer = textPipeline.Fit(emptyDataView);

In [ ]:
public class TransformedTextData : TextData
{
    public float[] Features { get; set; }
}

var predictionEngine = mlContext.Model.CreatePredictionEngine<TextData, TransformedTextData>(textTransformer);

In [ ]:
var data = new TextData() { Text = "This is a great product. I would like to buy it again." };
var prediction = predictionEngine.Predict(data);

In [ ]:
Console.WriteLine($"Number of Features: {prediction.Features.Length}");
Console.Write("Features: ");
foreach (var f in prediction.Features)
    Console.Write($"{f:F4} ");